In [1]:
import torch
import pandas as pd
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel, get_linear_schedule_with_warmup
from torch.optim import AdamW

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk 
import joblib
import os
import re
from tqdm import tqdm

2025-10-07 14:11:52.196460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759846312.445253      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759846312.509935      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
MODEL_NAME = 'gpt2'

USER_A_TOKEN = "<|user_a|>"
USER_B_TOKEN = "<|user_b|>"

In [3]:
DATA_PATH = '/Desktop/Dataset/conversationfile.xlsx'

In [4]:
CONTEXT_WINDOW = 12

EPOCHS = 100
LEARNING_RATE = 5e-5
BATCH_SIZE = 4
MAX_LEN = 256

In [5]:
OUTPUT_DIR = './submission_folder/'
MODEL_SAVE_PATH = os.path.join(OUTPUT_DIR, 'Model.joblib')

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [6]:
def load_and_prepare_data(file_path):
    try:
        df = pd.read_excel(file_path)
        
        df.columns = [col.strip() for col in df.columns]
        
        required_cols = ['Conversation ID', 'Timestamp', 'Sender', 'Message']
        for col in required_cols:
            if col not in df.columns:
                raise ValueError(f"Missing required column: {col}")
        
        # Clean data: remove rows with empty messages and strip whitespace
        df.dropna(subset=['Message'], inplace=True)
        df['Message'] = df['Message'].astype(str).str.strip()
        df['Sender'] = df['Sender'].str.strip()
        
        # Sort values to ensure chronological order within each conversation
        df.sort_values(by=['Conversation ID', 'Timestamp'], inplace=True)
        
        print("Data loaded and prepared successfully.")
        return df.reset_index(drop=True)
        
    except FileNotFoundError:
        print(f"Error: The file was not found at {file_path}")
        print("Please ensure the DATA_PATH in Section 2 is correct.")
        return None
    except Exception as e:
        print(f"An error occurred while loading data: {e}")
        return None

In [7]:
def create_training_examples(df, tokenizer, context_window):
    training_texts = []
    for convo_id, group in df.groupby('Conversation ID'):
        messages = group['Message'].tolist()
        senders = group['Sender'].tolist()
        
        for i in range(1, len(messages)):
            if senders[i] == 'User A' and senders[i-1] == 'User B':
                # Context is the preceding messages in the *same conversation*.
                start_index = max(0, i - context_window)
                context_slice = list(zip(senders[start_index:i], messages[start_index:i]))
                
                prompt = ""
                for sender, message in context_slice:
                    sender_token = USER_A_TOKEN if sender == 'User A' else USER_B_TOKEN
                    prompt += f"{sender_token} {message} "
                
                # The target reply is the current message from User A.
                reply = f"{USER_A_TOKEN} {messages[i]}"
                
                # Combine into a single string for the language model.
                training_texts.append(f"{prompt.strip()} {reply.strip()} {tokenizer.eos_token}")

    print(f"Created {len(training_texts)} training examples.")
    return training_texts

In [8]:
print("--- Starting Data Processing ---")
conversation_df = load_and_prepare_data(DATA_PATH)

if conversation_df is None:
    raise SystemExit("Stopping execution due to data loading failure.")

--- Starting Data Processing ---
Data loaded and prepared successfully.


In [9]:
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

In [10]:
special_tokens_dict = {'additional_special_tokens': [USER_A_TOKEN, USER_B_TOKEN], 'pad_token': '<|pad|>'}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
print("Tokenizer and Model loaded and configured.")

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Tokenizer and Model loaded and configured.


In [11]:
training_data = create_training_examples(conversation_df, tokenizer, CONTEXT_WINDOW)
if not training_data:
    raise ValueError("No valid training examples could be created. Please check the dataset for conversations where User A replies to User B.")

Created 9 training examples.


In [12]:
train_texts, val_texts = train_test_split(training_data, test_size=0.1, random_state=42)

In [13]:
class ChatDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.examples = []
        for text in tqdm(texts, desc="Tokenizing data"):
            tokenized_output = self.tokenizer(
                text,
                truncation=True,
                max_length=self.max_length,
                padding="max_length",
                return_tensors="pt"
            )
            self.examples.append(tokenized_output)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        item = self.examples[idx]
        # For a Causal Language Model like GPT-2, labels are the same as input_ids.
        return {
            "input_ids": item['input_ids'].flatten(),
            "attention_mask": item['attention_mask'].flatten(),
            "labels": item['input_ids'].flatten() 
        }

print("\n--- Creating Datasets and Dataloaders ---")
train_dataset = ChatDataset(train_texts, tokenizer, MAX_LEN)
val_dataset = ChatDataset(val_texts, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
print("DataLoaders are ready.")


--- Creating Datasets and Dataloaders ---


Tokenizing data: 100%|██████████| 1/1 [00:00<00:00, 791.08it/s]

DataLoaders are ready.


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"\nModel moved to device: {device}")

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


Model moved to device: cuda


In [15]:
print("--- Starting Model Fine-Tuning ---")
for epoch in range(EPOCHS):
    print(f"\n--- Epoch {epoch + 1}/{EPOCHS} ---")
    model.train()
    total_train_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        model.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Average Training Loss for Epoch {epoch+1}: {avg_train_loss:.4f}")

print("\nFine-tuning complete.")

--- Starting Model Fine-Tuning ---

--- Epoch 1/100 ---


Training Epoch 1: 100%|██████████| 2/2 [00:01<00:00,  1.88it/s]


Average Training Loss for Epoch 1: 10.7743

--- Epoch 2/100 ---


Training Epoch 2: 100%|██████████| 2/2 [00:00<00:00,  4.94it/s]


Average Training Loss for Epoch 2: 9.9142

--- Epoch 3/100 ---


...

Average Training Loss for Epoch 98: 0.1999

--- Epoch 99/100 ---


Training Epoch 99: 100%|██████████| 2/2 [00:00<00:00,  4.97it/s]


Average Training Loss for Epoch 99: 0.1945

--- Epoch 100/100 ---


Training Epoch 100: 100%|██████████| 2/2 [00:00<00:00,  4.95it/s]

Average Training Loss for Epoch 100: 0.1992

Fine-tuning complete.


In [16]:
model_and_tokenizer_to_save = {
    'model_state_dict': model.to('cpu').state_dict(),
    'tokenizer': tokenizer,
    'model_name': MODEL_NAME
}

joblib.dump(model_and_tokenizer_to_save, MODEL_SAVE_PATH)
print(f"Model and tokenizer saved to {MODEL_SAVE_PATH}")

Model and tokenizer saved to ./submission_folder/Model.joblib


In [17]:
def load_model_from_joblib(path):
    saved_data = joblib.load(path)
    
    tokenizer = saved_data['tokenizer']
    model = GPT2LMHeadModel.from_pretrained(saved_data['model_name'])
    model.resize_token_embeddings(len(tokenizer))
    model.load_state_dict(saved_data['model_state_dict'])
    
    return model, tokenizer

In [18]:
def generate_reply(model, tokenizer, history_df, new_message_from_b, device):
    model.to(device)
    model.eval()

    prompt = ""
    for _, row in history_df.iterrows():
        sender_token = USER_A_TOKEN if row['Sender'] == 'User A' else USER_B_TOKEN
        prompt += f"{sender_token} {row['Message']} "
    
    prompt += f"{USER_B_TOKEN} {new_message_from_b} {USER_A_TOKEN}"

    inputs = tokenizer(prompt, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,  # Limit the length of the new reply
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2,
            do_sample=True,     # Activate sampling
            top_k=50,           # Consider the top 50 tokens
            top_p=0.95,         # Use nucleus sampling
        )
    
    generated_tokens = outputs[0][inputs['input_ids'].shape[-1]:]
    reply = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    return reply

In [24]:
print("\n--- Testing Reply Generation ---")
# Load the model back for a clean test
loaded_model, loaded_tokenizer = load_model_from_joblib(MODEL_SAVE_PATH)

# [cite_start]Context from Conversation ID 4 [cite: 1]
conversation_history = pd.DataFrame([
    {'Sender': 'User A', 'Message': "Finally watched that new sci-fi movie everyone's talking about."},
    {'Sender': 'User B', 'Message': "Nice! What did you think? I loved the visuals."},
    {'Sender': 'User A', 'Message': "Visuals were amazing, but the plot was a bit predictable for me."}
])
user_b_new_message = "I can see that. The ending felt a bit rushed. Still a fun watch though."

generated_reply = generate_reply(loaded_model, loaded_tokenizer, conversation_history, user_b_new_message, device)
print(f"Conversation Context:\n{conversation_history.to_string(index=False)}\n")
print(f"User B says: '{user_b_new_message}'")
print(f"Generated User A reply: '{generated_reply}'")
print(f"Actual User A reply from data: 'Definitely. Worth it just for the big screen experience.'")


--- Testing Reply Generation ---
Conversation Context:
Sender                                                          Message
User A  Finally watched that new sci-fi movie everyone's talking about.
User B                   Nice! What did you think? I loved the visuals.
User A Visuals were amazing, but the plot was a bit predictable for me.

User B says: 'I can see that. The ending felt a bit rushed. Still a fun watch though.'
Generated User A reply: '"Definitely. Worth it just for the big screen experience."'
Actual User A reply from data: 'Definitely. Worth it just for the big screen experience.'


In [25]:
def evaluate_model(model, val_loader, tokenizer, device):
    model.to(device)
    model.eval()
    
    total_perplexity_loss = 0
    bleu_scores = []
    chencherry = SmoothingFunction()
    
    print("\n--- Starting Final Evaluation ---")
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Calculating Perplexity"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            total_perplexity_loss += outputs.loss.item()
            
        avg_loss = total_perplexity_loss / len(val_loader)
        perplexity = torch.exp(torch.tensor(avg_loss))
        print(f"  - Perplexity on Validation Set: {perplexity.item():.4f}")

        for text in tqdm(val_texts, desc="Calculating BLEU Score"):
            prompt_end_index = text.rfind(USER_A_TOKEN)
            if prompt_end_index == -1: continue

            prompt = text[:prompt_end_index] + USER_A_TOKEN
            reference_reply = text[prompt_end_index:].replace(USER_A_TOKEN, "").replace(tokenizer.eos_token, "").strip()
            
            inputs = tokenizer(prompt, return_tensors='pt').to(device)
            
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                top_k=50,
                top_p=0.95,
            )
            
            generated_tokens = outputs[0][inputs['input_ids'].shape[-1]:]
            generated_reply = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

            reference_tokens = [reference_reply.split()]
            candidate_tokens = generated_reply.split()
            
            if candidate_tokens:
                bleu_score = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=chencherry.method1)
                bleu_scores.append(bleu_score)
            
    avg_bleu = np.mean(bleu_scores) if bleu_scores else 0
    print(f"  - Average BLEU Score on Validation Set: {avg_bleu:.4f}")
    
    return perplexity.item(), avg_bleu

In [26]:
final_perplexity, final_bleu = evaluate_model(loaded_model, val_loader, loaded_tokenizer, device)

print("\n--- Script Finished Successfully ---")


--- Starting Final Evaluation ---


Calculating Perplexity: 100%|██████████| 1/1 [00:00<00:00, 58.90it/s]


  - Perplexity on Validation Set: 1.1501


Calculating BLEU Score: 100%|██████████| 1/1 [00:00<00:00,  2.56it/s]

  - Average BLEU Score on Validation Set: 0.4098

--- Script Finished Successfully ---
